In [40]:
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.utils import negative_sampling
import torch
from torch_geometric.nn.conv import GCNConv
from torch_geometric.data import Data
import torch_geometric.transforms as T

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

dataset = "icews18"

In [41]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

In [42]:
@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy()), average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [43]:
def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=50
):
    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc, val_ap = eval_link_predictor(model, val_data)

        if epoch % 10 == 0:
            print(f"Epoch: {epoch:06d}, Train Loss: {loss:.6f}, Val AUC: {val_auc:.6f}, Val AP: {val_ap:.6f}")

    

    return model

In [44]:
def create_edge_index_label(x, edge_index, data):
    pos_edge_index = edge_index.clone()
    num_pos = pos_edge_index.size(1)
    pos_edge_label = torch.ones(num_pos, dtype=torch.float)

    neg_edge_index = negative_sampling(
        edge_index=edge_index,
        num_nodes=x.size(0),
        num_neg_samples=num_pos,
        method='sparse'
    )
    neg_edge_label = torch.zeros(neg_edge_index.size(1), dtype=torch.float)

    data.edge_label_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)
    data.edge_label = torch.cat([pos_edge_label, neg_edge_label], dim=0)


def create_data(df, train_idx, val_idx, test_idx):
    head1,head2 = df.columns[:2]
    edge_index = torch.tensor(df.iloc[train_idx][[head1, head2]].values.T, dtype=torch.long)
    x = torch.randn(edge_index.max().item()+1, 512)
    train_data = Data(edge_index=edge_index, x=x)
    create_edge_index_label(x, edge_index, train_data)

    edge_index = torch.tensor(df.iloc[val_idx][[head1, head2]].values.T, dtype=torch.long)
    val_data = Data(edge_index=edge_index, x=x)
    create_edge_index_label(x, edge_index, val_data)

    edge_index = torch.tensor(df.iloc[test_idx][[head1, head2]].values.T, dtype=torch.long)
    test_data = Data(edge_index=edge_index,x=x)
    create_edge_index_label(x, edge_index, test_data)

    return train_data, val_data, test_data
    

In [45]:
user_artist = pd.read_csv(f'../../magnn/{dataset}/data/raw/{dataset}/1-indexed/actor_actor.csv', encoding='utf-8', names=['userID','artistID', 'weight'],)
user_friend = pd.read_csv(f'../../magnn/{dataset}/data/raw/{dataset}/1-indexed/actor_action.csv', encoding='utf-8', names=['userID', 'friendID'])
artist_tag = pd.read_csv(f'../../magnn/{dataset}/data/raw/{dataset}/1-indexed/actor_sector.csv', encoding='utf-8', names=['artistID', 'tagID'])

num_user = user_artist['userID'].max()
num_artist = user_artist['artistID'].max()
num_tag = artist_tag['tagID'].max()

indices = np.arange(len(user_artist))
train_idx, test_idx = train_test_split(indices, test_size=0.15, random_state=42)
val_idx, test_idx = train_test_split(test_idx, test_size=0.5, random_state=42)

train_data, val_data, test_data = create_data(user_artist, train_idx, val_idx, test_idx)


In [46]:
####   If you want to split train edges from val and test edges


# head1,head2 = user_artist.columns[:2]
# edge_index = torch.tensor(user_artist.iloc[train_idx][[head1, head2]].values.T, dtype=torch.long)

# edge_index

# x = torch.randn(edge_index.max().item(), 512)
# graph = Data(edge_index=edge_index, x=x)
# split = T.RandomLinkSplit(
#     num_val=0.05,
#     num_test=0.1,
#     is_undirected=True,
#     add_negative_train_samples=False,
#     neg_sampling_ratio=1.0,
# )

# train_data, val_data, test_data = split(graph)

In [47]:
model = Net(512, 256, 128).to('cuda')
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

model = train_link_predictor(model, train_data.to('cuda'), val_data.to('cuda'), optimizer, criterion)

Epoch: 000010, Train Loss: 1.444249, Val AUC: 0.664702, Val AP: 0.646749
Epoch: 000020, Train Loss: 0.851542, Val AUC: 0.808316, Val AP: 0.796695
Epoch: 000030, Train Loss: 0.690397, Val AUC: 0.850479, Val AP: 0.846448
Epoch: 000040, Train Loss: 0.637123, Val AUC: 0.857294, Val AP: 0.858649
Epoch: 000050, Train Loss: 0.611542, Val AUC: 0.860772, Val AP: 0.864943


In [48]:
test_auc, test_ap = eval_link_predictor(model, test_data.to('cuda'))

print(f"Test: AUC : {test_auc:.6f}, AP : {test_ap:.6f}")

Test: AUC : 0.864634, AP : 0.869105
